In [1]:
import logging

import extract



logging.basicConfig(format='%(asctime)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

params = {"codeCommuneEtablissement": "*","etatAdministratifUniteLegale": "A",
'categorieJuridiqueUniteLegale': "5*",
'activitePrincipaleUniteLegale': "16.10A", 'etablissementSiege':True}
ts=extract.query(logger, params)

#display(ts) 



2020-11-25 11:11:32,375 
QUERY                          :
codeCommuneEtablissement       : *
etatAdministratifUniteLegale   : A
categorieJuridiqueUniteLegale  : 5*
activitePrincipaleUniteLegale  : 16.10A
etablissementSiege             : True
2020-11-25 11:11:32,378 File not available --> API call
2020-11-25 11:11:33,751 ... total : 1804
2020-11-25 11:12:22,368 ... +1000 = 1000/1804
2020-11-25 11:12:29,587 ... +804 = 1804/1804
2020-11-25 11:12:30,490 ... +0 = 1804/1804
/home/ec2-user/dev/sireneTools/env/lib64/python3.7/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (1,15,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
2020-11-25 11:12:40,578 - - - - - - -
Nb entreprises                : 1805 
Fichier                       : files/617f68670b3967dc6507851debbfb736.data


In [ ]:
ts.dtypes

In [ ]:
import pandas as pd
si= pd.read_csv("chiffres-cles-2019_16.10A.csv",sep=";")
si['isData'] = 'Y'
si = si.astype({"Siren":  str})
si.reset_index(inplace=True)
#siShort = si[['Identifiant de l\'entreprise', 'isData']]
#siShort.reset_index(inplace=True)



In [ ]:
len(si.index)


In [2]:
ts.isnull().sum()

siren                   0
siret                   0
activite                0
activiteLong            0
NAF 732                 0
INTITULÉS 732           0
categorie             264
effectif              314
commune                 0
catJuridique            0
denomination            0
departement             0
CA 1                 1558
CA 2                 1551
CA 3                 1524
etatAdministratif       0
dtype: int64

In [ ]:
ts_comp=pd.merge(ts, si[['Siren','Code APE','CA 1','CA 2', 'CA 3']], left_on='siren', right_on='Siren', how='outer')
ts_comp["siren"].fillna("EMPTY", inplace = True)
len(ts_comp.index)

In [ ]:
ts_comp.isnull().sum()

In [ ]:
ts_comp[ts_comp.Siren.isnull()]

In [ ]:
len(si.index)


In [ ]:
ts_comp.groupby(['siren' ]).size()


In [ ]:
ts.columns


In [ ]:
ts_si=pd.merge(ts, siShort, left_on='siren',right_on='Identifiant de l\'entreprise', how='outer')

len(ts_si.index)

In [ ]:
def UV(u, v):
    if u==u and v==v:
        return 'UV'
    else:
        if u == u  and not v ==v:
             return 'UNotV'
        else:
            if not u == u  and not v ==v:
                return 'NotUNotV'
            return 'NotUV'
def U(u):
    if u==u:
        return "Y"
    else:
        return "N"


ts_comp['CA1YN'] = list(map(UV, ts_comp['CA 1_x'], ts_comp['CA 1_y']))


ts_comp.reset_index(drop=True, inplace=True)

len(ts_comp.index)

In [ ]:
ts_comp.groupby(['CA1YN' ]).size()


In [ ]:
ts_comp[ts_comp['CA1YN']=='NotUV']

In [ ]:
import pandas as pd
piv=pd.pivot_table(ts_si,index=['etatAdministratif','catJ'], columns=['SI'], values=["catJuridique"], aggfunc='count', margins=True,fill_value=0)

piv

In [ ]:
catFilter = ['5']
etatFilter = ['A']
SIFilter=['N']
ts_si[ts_si.catJ.isin(catFilter) & ts_si.etatAdministratif.isin(etatFilter) & ts_si.SI.isin(SIFilter) ]


In [ ]:
import pandas as pd
piv=pd.pivot_table(ts_si,index=['etatAdministratif'],columns=['UV'], values=["index"], aggfunc='count')

piv

In [ ]:
print(ts['CA 3'].isnull().sum())
print(ts['CA 3'].notnull().sum())

In [ ]:
import pandas as pd

labels = ["{0} - {1}".format(i, i + 9) for i in range(0, 100, 1000000)]
ts['group'] = pd.cut(ts["CA 1"], range(0, 10000000, 1000000), right=False,)

pd.pivot_table(ts,index=['group'],values=["siret"], aggfunc='count')